# 分层索引

In [1]:
import pandas as pd
import numpy as np 

In [2]:
data1 = pd.Series(np.random.randn(9), index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'], [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data1

a  1   -2.365804
   2    0.716790
   3   -0.762144
b  1    1.084328
   3   -0.421314
c  1   -0.567264
   2   -0.674697
d  2   -1.140305
   3    0.850788
dtype: float64

In [3]:
data1.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [4]:
data1['b']

1    1.084328
3   -0.421314
dtype: float64

In [5]:
data1.loc['b':'c']

b  1    1.084328
   3   -0.421314
c  1   -0.567264
   2   -0.674697
dtype: float64

In [6]:
data1.loc[['b', 'd']]

b  1    1.084328
   3   -0.421314
d  2   -1.140305
   3    0.850788
dtype: float64

In [7]:
data1.loc[:, 2]

a    0.716790
c   -0.674697
d   -1.140305
dtype: float64

In [8]:
data1.unstack().stack()

a  1   -2.365804
   2    0.716790
   3   -0.762144
b  1    1.084328
   3   -0.421314
c  1   -0.567264
   2   -0.674697
d  2   -1.140305
   3    0.850788
dtype: float64

In [9]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)), index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]], columns=[['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [10]:
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [11]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

## 重排序与层级排序

In [12]:
frame.swaplevel(0, 1)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [13]:
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

## 按层级进行汇总统计

In [14]:
frame.sum(level=1) # 等价于frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [15]:
frame.sum(level='state', axis=1)

state      Ohio  Colorado
key1 key2                
a    1        1         2
     2        7         5
b    1       13         8
     2       19        11

## 使用列进行索引

In [16]:
df = pd.DataFrame({'a':range(7), 'b':range(7,0,-1), 'c':['one', 'one', 'one', 'two', 'two', 'two', 'two'], 'd':[0, 1, 2, 0, 1, 2, 3]})
df

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [17]:
df.set_index(['c', 'd'])

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [18]:
df.set_index(['c', 'd']).reset_index() # reset_index()是set_index()的反操作

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


# 联合与合并数据集

## 数据库风格的DataFrame连接

In [19]:
df1 = pd.DataFrame({'key':['b', 'b', 'a', 'c', 'a', 'a', 'b'], 'data1':range(7)})
df2 = pd.DataFrame({'key':['a', 'b', 'd'], 'data2':range(3)})

In [20]:
pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [21]:
df3 = pd.DataFrame({'lkey':['b', 'b', 'a', 'c', 'a', 'a', 'b'], 'data1':range(7)})
df4 = pd.DataFrame({'rkey':['a', 'b', 'd'], 'data2':range(3)})

In [22]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey', how='outer') # 使用双方的键进行连接

,lkey,data1,rkey,data2
0,b,0.0,b,1.0
1,b,1.0,b,1.0
2,b,6.0,b,1.0
3,a,2.0,a,0.0
4,a,4.0,a,0.0
5,a,5.0,a,0.0
6,c,3.0,NaN,NaN
7,NaN,NaN,d,2.0


In [23]:
left = pd.DataFrame({'key1':['foo', 'foo', 'bar'], 'key2':['one', 'two', 'one'], 'lval':[1, 2, 3]})
right = pd.DataFrame({'key1':['foo', 'foo', 'bar', 'bar'], 'key2':['one', 'one', 'one', 'two'], 'rval':[4, 5, 6, 7]})
pd.merge(left, right, on=['key1', 'key2'], how='outer') # 使用多个键进行合并

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [24]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right')) # 解决重复列明的问题

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


## 沿轴向连接

In [25]:
arr = np.arange(12).reshape((3, 4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [26]:
np.concatenate([arr, arr], axis=0)

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [27]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])

pd.concat([s1, s2, s3], axis=1, join='outer')

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [28]:
pd.concat([s1, s2, s3], keys=['one', 'two', 'three']) # 利用keys在0轴上创建一个多层索引

one    a    0
       b    1
two    c    2
       d    3
       e    4
three  f    5
       g    6
dtype: int64

In [29]:
pd.concat([s1, s2, s3], keys=['one', 'two', 'three'], axis=1) # 利用keys在1轴上修改（增加）了列头

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [30]:
df5 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'], columns=['one', 'two'])
df6 = pd.DataFrame(np.arange(4).reshape(2, 2), index=['a', 'c'], columns=['three', 'four'])
pd.concat([df5, df6], axis=1, keys=['level1', 'level2'])

level1     level2     
     one two  three four
a      0   1    0.0  1.0
b      2   3    NaN  NaN
c      4   5    2.0  3.0

In [31]:
pd.concat({'level1':df5, 'level2':df6}, axis=1) # 利用字典也可创建

level1     level2     
     one two  three four
a      0   1    0.0  1.0
b      2   3    NaN  NaN
c      4   5    2.0  3.0

In [32]:
pd.concat({'level1':df5, 'level2':df6}, axis=0, names=['upper', 'lower'])

one  two  three  four
upper  lower                       
level1 a      0.0  1.0    NaN   NaN
       b      2.0  3.0    NaN   NaN
       c      4.0  5.0    NaN   NaN
level2 a      NaN  NaN    0.0   1.0
       c      NaN  NaN    2.0   3.0

In [33]:
pd.concat([df5, df6], ignore_index=True) # ignore_index=True时抛弃被合并数据的索引并重新生成共同的初始索引，默认为False

,one,two,three,four
0,0.0,1.0,NaN,NaN
1,2.0,3.0,NaN,NaN
2,4.0,5.0,NaN,NaN
3,NaN,NaN,0.0,1.0
4,NaN,NaN,2.0,3.0


## 联合重叠数据

In [34]:
a = pd.Series([np.nan, 2.5, 0, 3.5, 4.5, np.nan], index=['f', 'e', 'd', 'c', 'b', 'a'])
b = pd.Series([0, np.nan, 2, np.nan, np.nan, 5], index=['a', 'b', 'c', 'd', 'e', 'f'])
np.where(pd.isnull(a), b, a)

array([0. , 2.5, 0. , 3.5, 4.5, 5. ])

In [35]:
df7 = pd.DataFrame({'a':[1, np.nan, 5, np.nan], 'b':[np.nan, 2, np.nan, 6], 'c':range(2, 18, 4)})
df8 = pd.DataFrame({'a':[5, 4, np.nan, 3, 7], 'b':[np.nan, 3, 4, 6, 8]})
df7.combine_first(df8) # 若df7中元素不为na，则用df7的元素，否则使用df8的元素进行填补

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


# 重塑与透视

## 使用索引进行重塑

In [36]:
data2 = pd.DataFrame(np.arange(6).reshape((2, 3)), index=pd.Index(['Ohio', 'Colorado'], name='state'), columns=pd.Index(['one', 'two', 'three'], name='number'))
result = data2.stack()
result.unstack(1) # stack()中的序号代表索引层数，最外层为第0层。参数也可以为多重索引中索引的名称，比如这一行代码可以改为result.unstack('number')

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [37]:
result.unstack('number')

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [38]:
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [39]:
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


## 将“长”透视为“宽”

In [40]:
data3 = pd.read_excel(r'C:\Users\skaha\Desktop\git\python数据分析\数据规整：连接、联合与重塑\可用rsr数据母表.xlsx').iloc[:100, :10]

In [48]:
pd.pivot_table(data3, index=['省份', '州市'], columns=['年份'], values=['财政支出增长率'])

财政支出增长率                                        
年份                    2015      2016      2017      2018      2019
省份     州市                                                         
内蒙古自治区 包头市        0.111899  0.053625       NaN       NaN       NaN
       赤峰市        0.151453  0.057377  0.053909       NaN       NaN
       阿拉善盟      -0.008350  0.083952 -0.080577  0.160765  0.146360
北京市    北京市        0.268093  0.116592  0.064436  0.095026 -0.007938
四川省    德阳市        0.113575  0.034901  0.057406  0.132310  0.059738
       成都市        0.095836  0.086808  0.100738  0.045975  0.092263
       达州市        0.128663  0.107241       NaN       NaN  0.003104
       阿坝藏族羌族自治州       NaN       NaN  0.093225  0.221022  0.036609
安徽省    安庆市        0.125030  0.000742  0.106336  0.126550  0.103203
       池州市        0.063015  0.009675 -0.036169  0.075373  0.065846
       蚌埠市        0.173380       NaN       NaN       NaN       NaN
山东省    东营市        0.067460  0.040783       NaN       NaN       NaN
       德州市             NaN       NaN       NaN       NaN  0.056052
山西省    大同市        0.311176 -0.016899  0.138574  0.026068  0.087943
广东省    东莞市        0.269968  0.031052  0.114069  0.146417  0.127525
江苏省    常州市        0.115884  0.046937  0.085500  0.078445  0.099812
河北省    保定市        0.238654  0.079842  0.173117  0.463027 -0.248842
       承德市        0.150265  0.038344  0.111555  0.117071  0.118404
       沧州市        0.271616  0.041392  0.090281  0.130381  0.102933
湖南省    常德市        0.175735       NaN       NaN       NaN  0.110225
辽宁省    丹东市        0.035550 -0.122943  0.072617  0.077310  0.089709
       大连市       -0.079602 -0.044377  0.056950  0.088765  0.014773
       朝阳市       -0.148730  0.108752  0.089444  0.143682  0.121689
       本溪市             NaN       NaN -0.014764  0.116882       NaN
       鞍山市       -0.248707  0.022277  0.095704  0.012980  0.178761
陕西省    安康市             NaN       NaN       NaN       NaN  0.153476

In [50]:
data3.pivot(index=['省份', '州市'], columns=['年份'], values=['财政支出增长率'])

财政支出增长率                                        
年份                    2015      2016      2017      2018      2019
省份     州市                                                         
内蒙古自治区 包头市        0.111899  0.053625       NaN       NaN       NaN
       赤峰市        0.151453  0.057377  0.053909       NaN       NaN
       阿拉善盟      -0.008350  0.083952 -0.080577  0.160765  0.146360
北京市    北京市        0.268093  0.116592  0.064436  0.095026 -0.007938
四川省    德阳市        0.113575  0.034901  0.057406  0.132310  0.059738
       成都市        0.095836  0.086808  0.100738  0.045975  0.092263
       达州市        0.128663  0.107241       NaN       NaN  0.003104
       阿坝藏族羌族自治州       NaN       NaN  0.093225  0.221022  0.036609
安徽省    安庆市        0.125030  0.000742  0.106336  0.126550  0.103203
       池州市        0.063015  0.009675 -0.036169  0.075373  0.065846
       蚌埠市        0.173380       NaN       NaN       NaN       NaN
山东省    东营市        0.067460  0.040783       NaN       NaN       NaN
       德州市             NaN       NaN       NaN       NaN  0.056052
山西省    大同市        0.311176 -0.016899  0.138574  0.026068  0.087943
广东省    东莞市        0.269968  0.031052  0.114069  0.146417  0.127525
江苏省    常州市        0.115884  0.046937  0.085500  0.078445  0.099812
河北省    保定市        0.238654  0.079842  0.173117  0.463027 -0.248842
       承德市        0.150265  0.038344  0.111555  0.117071  0.118404
       沧州市        0.271616  0.041392  0.090281  0.130381  0.102933
湖南省    常德市        0.175735       NaN       NaN       NaN  0.110225
辽宁省    丹东市        0.035550 -0.122943  0.072617  0.077310  0.089709
       大连市       -0.079602 -0.044377  0.056950  0.088765  0.014773
       朝阳市       -0.148730  0.108752  0.089444  0.143682  0.121689
       本溪市             NaN       NaN -0.014764  0.116882       NaN
       鞍山市       -0.248707  0.022277  0.095704  0.012980  0.178761
陕西省    安康市             NaN       NaN       NaN       NaN  0.153476

In [51]:
data3.pivot_table(index=['省份', '州市'], columns=['年份'], values='财政支出增长率') # 尽量使用这个函数，而不是pivot()

年份                    2015      2016      2017      2018      2019
省份     州市                                                         
内蒙古自治区 包头市        0.111899  0.053625       NaN       NaN       NaN
       赤峰市        0.151453  0.057377  0.053909       NaN       NaN
       阿拉善盟      -0.008350  0.083952 -0.080577  0.160765  0.146360
北京市    北京市        0.268093  0.116592  0.064436  0.095026 -0.007938
四川省    德阳市        0.113575  0.034901  0.057406  0.132310  0.059738
       成都市        0.095836  0.086808  0.100738  0.045975  0.092263
       达州市        0.128663  0.107241       NaN       NaN  0.003104
       阿坝藏族羌族自治州       NaN       NaN  0.093225  0.221022  0.036609
安徽省    安庆市        0.125030  0.000742  0.106336  0.126550  0.103203
       池州市        0.063015  0.009675 -0.036169  0.075373  0.065846
       蚌埠市        0.173380       NaN       NaN       NaN       NaN
山东省    东营市        0.067460  0.040783       NaN       NaN       NaN
       德州市             NaN       NaN       NaN       NaN  0.056052
山西省    大同市        0.311176 -0.016899  0.138574  0.026068  0.087943
广东省    东莞市        0.269968  0.031052  0.114069  0.146417  0.127525
江苏省    常州市        0.115884  0.046937  0.085500  0.078445  0.099812
河北省    保定市        0.238654  0.079842  0.173117  0.463027 -0.248842
       承德市        0.150265  0.038344  0.111555  0.117071  0.118404
       沧州市        0.271616  0.041392  0.090281  0.130381  0.102933
湖南省    常德市        0.175735       NaN       NaN       NaN  0.110225
辽宁省    丹东市        0.035550 -0.122943  0.072617  0.077310  0.089709
       大连市       -0.079602 -0.044377  0.056950  0.088765  0.014773
       朝阳市       -0.148730  0.108752  0.089444  0.143682  0.121689
       本溪市             NaN       NaN -0.014764  0.116882       NaN
       鞍山市       -0.248707  0.022277  0.095704  0.012980  0.178761
陕西省    安康市             NaN       NaN       NaN       NaN  0.153476